# Multi-layer Artificial Neural Network

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset
from feature_engineering import process_categorical_columns


In [2]:

data = pd.read_csv('StudentPerformanceFactorsCleaned.csv')

X = data.drop(columns=['Exam_Score'])
y = data['Exam_Score']













In [3]:
# Encode the categrorical data numerically and perform a train-val-test split and stndardize based on the training set
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first')


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=21)

# Apply transformations
X_train_transformed = preprocessor.fit_transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

In [4]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_transformed, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_transformed, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_transformed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [6]:
# Assuming X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor are already defined
# Define model
input_size = X_train_tensor.shape[1]
model = MLP(input_size)

# Define loss and optimizer
# We are doing regressions, so we are using MSE for the loss function
criterion = nn.MSELoss()
# Several values of the learning rate were tried and this one worked the best
# We also tried 0.01, and 0.005, and 0.0001, but this one was best
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop without DataLoader
# Use a fairly standard option of batch size
batch_size = 32
# We picked an epoch number large enough to get to a very small loss but not so
# large that it would take too long to train. We tried a few values out to find
# something that would have this effect and settled on 2000. We started at 100,
# but that was too small, so it was increased to 500 and later 2000. We could tell
# that going beyond that would take too long, so 2000 was ultimately what we used.
epochs = 2000
# Patience was chosen by trial and error, starting with 20 and changing based
# on the fact that a test revealed that validation loss improved several
# times afterwards. 35 seems about right, getting a pretty good validation loss.
# Tried 10, 20 and 100 as well. 20 worked well enough, but 35 was a bit better.
# 10 was underfitting and 100 was overfitting.
MAX_PATIENCE = 35
best_val_loss = float('inf')
patience = MAX_PATIENCE

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    # Manually create mini-batches
    for i in range(0, len(X_train_tensor), batch_size):
        # Select a batch
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Compute the loss on the validation set
    inputs = X_val_tensor
    outputs = model(inputs)
    targets = y_val_tensor
    val_loss = criterion(outputs, targets)
    if val_loss < best_val_loss:
        patience = MAX_PATIENCE
        best_val_loss = val_loss
    else:
        patience -= 1
        if patience == 0:
            print(f"Stopping early at epoch {epoch+1}.")
            break

    # Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss / (len(X_train_tensor) // batch_size):.4f}')

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
    y_true = y_test_tensor.numpy()

mse = mean_squared_error(y_true, y_pred)
print(f'Mean Squared Error on Test Data: {mse}')

Epoch 100/2000, Loss: 57.5209
Epoch 200/2000, Loss: 44.1253
Epoch 300/2000, Loss: 33.5123
Stopping early at epoch 318.
Mean Squared Error on Test Data: 3.8128578662872314
